# Swallowつかってみよう

公式ページ
https://tokyotech-llm.github.io/

Huggingface(70bのやつ)
https://huggingface.co/tokyotech-llm/Swallow-70b-instruct-hf

ブログ記事（詳細情報など）
https://zenn.dev/tokyotech_lm/articles/d6cb3a8fdfc907

使うときはRuntime をA100などにする。
CPU

## Huggingfaceに書いてあるやつそのままやってみたよ

In [ ]:
!pip install torch transformers sentencepiece accelerate protobuf langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 他にも13bと70bのパラメータで作ったモデルがある。tokyotech-llm/Swallow-13b-instruct-hf とかSwallow-70b-instruct-hfとか
# 7bだとモデルのロードがすぐ終わるけど…
model_name = "tokyotech-llm/Swallow-7b-instruct-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, device_map="auto")


PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"

    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}

def create_prompt(instruction, input=None):
    """
    Generates a prompt based on the given instruction and an optional input.
    If input is provided, it uses the 'prompt_input' template from PROMPT_DICT.
    If no input is provided, it uses the 'prompt_no_input' template.

    Args:
        instruction (str): The instruction describing the task.
        input (str, optional): Additional input providing context for the task. Default is None.

    Returns:
        str: The generated prompt.
    """
    if input:
        # Use the 'prompt_input' template when additional input is provided
        return PROMPT_DICT["prompt_input"].format(instruction=instruction, input=input)
    else:
        # Use the 'prompt_no_input' template when no additional input is provided
        return PROMPT_DICT["prompt_no_input"].format(instruction=instruction)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
instruction_example = "以下のトピックに関する詳細な情報を提供してください。"
input_example = "日本の著作権保護期間は何年？"
prompt = create_prompt(instruction_example, input_example)
print(prompt)

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=500,
    temperature=0.99,
    top_p=0.95,
    do_sample=True,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
以下のトピックに関する詳細な情報を提供してください。

### 入力:
日本の著作権保護期間は何年？

### 応答:
以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
以下のトピックに関する詳細な情報を提供してください。

### 入力:
日本の著作権保護期間は何年？

### 応答:日本の著作権保護期間は70年間である。著作権者の死後70年まで著作権が認められ、その後、著作権保護は終了する。


## __なんとか応用できないものか__



# 注意
GPUの設定をA100にしないとRAM不足でエラーがでるかも。V100のRAM上限20GMだととまる…



[JaLCのAPI](https://api.japanlinkcenter.org/api-docs/index.html)でDOIのリストをゲット

[J-StageのAPI](https://www.jstage.jst.go.jp/static/pages/WebAPI/-char/ja)でもできる。
ただし論文要旨とか抄録とかの情報はどちらからも取れない…


ので、JaLCのAPIから例えば史学雑誌の新しい記事のDOIをゲットして、
URLからURLを取得してウェブページをスクレイピング



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


def getAbstract(doi):
    res = requests.get(doi)
    sp = BeautifulSoup(res.content)

    abst_tag = sp.find("meta", attrs={"name": "abstract"})
    if abst_tag is None:
        title_string = None
        abst_string = None
    else:
        abst_string = abst_tag["content"]
        title_string = sp.find("title").text

    return title_string, abst_string


jalc_link = "https://api.japanlinkcenter.org/doilist/10.24471?rows=50&page=1&sort=updated_date&order=desc"
re = requests.get(jalc_link)
json_data = re.json()

items = json_data["data"]["items"]
dois = []
for i in items:

    url = i["dois"]["url"]
    dois.append(url)

tit_list = []
abst_list = []
link_list = []
for j in dois:

    if "cover" in j:
        continue
    else:
        tit, abst = getAbstract(j)
        tit_list.append(tit)
        abst_list.append(abst)
        link_list.append(j)
        time.sleep(3)

df = pd.DataFrame(list(zip(tit_list, abst_list, link_list)), columns=["title", "abstract", "link"])
df_without_none = df.dropna()
#print(df_without_none)
df_without_none.to_csv("abstract_table.csv", encoding="utf-8", index=False)


In [ ]:
!pip install -U langchain-community faiss-cpu langchain-openai tiktoken cohere langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "自分のAPIをつかってね"

In [ ]:
import pandas as pd

from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

article_df = pd.read_csv("abstract_table.csv")
docs = []
for index, row in article_df.iterrows():

    title = row["title"]
    abst = row["abstract"]
    link = row["link"]

    metadata = {"article_title": title, "link": link}
    doc_obj = Document(page_content=abst, metadata=metadata)
    docs.append(doc_obj)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding=embeddings)
vectorstore.save_local("faiss_index")


## FAISSのベクトルストアをアップロードして使いたいとき


In [ ]:
# faissのベクトルストアをアップロードして使いたい時。faiss_indexというフォルダを作ってそこにindex.faissとindex.pklファイルをアップロードする
!mkdir faiss_index

faiss_indexのフォルダ下にデータをアップロードしてね。サンプルは[ここ](https://github.com/NbtKmy/swallow_trial)のリポジトリの"materials">"faiss_index"にあります。

In [ ]:
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local("faiss_index", embeddings)

In [ ]:
results_with_scores = vectorstore.similarity_search_with_score("中国の歴史")
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")
    metadata = doc.metadata
    #print(metadata["link"])

Content: 本稿は、七・八世紀の日本における啓という文書様式について、史料中の「啓」という語句の分析を手懸りに、中国における啓との比較を通じて検討したものである。 まず日本の史料をみると、「啓」は『日本書紀』本文や風土記では天皇以外に、『書紀』分註や天寿国繡帳銘では天皇にのみに用いる語句であり、ここからそれぞれの成立時期における啓の機能に差違がみいだせる。次に中国の啓は、六朝時代では公私問わず皇帝をも対象にできる汎用の上申文書（六朝的な啓）であり、唐代では皇帝以外に奉ずる文書（唐的な啓）であった。これにより、日本で天皇に「啓」を用いるのは六朝的な啓の影響で、天皇に「啓」を用いないのは唐的な啓の影響と考えられる。つまり、日本では六朝から唐的な啓へという転換が起きていたのである。また、東アジアにおける八世紀の啓をみると、啓は君主以外に奉ずる文書と言える。その機能に従って渤海は日本に啓を用いたが、日本は啓の機能を知りながら強引に渤海を朝貢国とする対応をとっており、日本の対渤海外交の方針が窺える。 　また、七世紀の文書木簡における啓は汎用の上申文書としての性格が窺え、これも六朝的な啓の影響が考えられる。啓は当時の状況に適した文書様式として受容され、文書行政を展開するための先駆的な役割を果たしていた。 このように七世紀は六朝的な啓が用いられていたが、大宝令施行によって唐的な啓への転換が起こり、公式令啓式条などにみえるように、啓は天皇以外に奉ずる文書となった。一方、公式令の範囲外では六朝の要素が残存しており、唐の啓と完全に同質となったわけでは無かった。とはいえ、啓の転換の結果、天皇に上申するための文書様式は、表・奏という天皇に上申するため「だけ」の文書様式となり、啓の転換は文書様式上における天皇の地位の可視化を意図していたと言えよう。, Metadata: {'article_title': '啓に関する基礎的考察', 'link': 'https://doi.org/10.24471/shigaku.131.10_1'}, Score: 0.37608426809310913
Content: 本稿では、日本外交史研究で「新外交」呼応策として言及されながら実証の不十分だった外務省による「宣伝外交」の実態を、ワシントン会議［華府会議］を例に解明する。そして、第一次世界大戦

In [ ]:
!pip install torch transformers sentencepiece accelerate protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.5 MB/s eta 0:00:00


## Langchain + Swallow

In [ ]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "tokyotech-llm/Swallow-7b-instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, device_map="auto")
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128
)
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from langchain import PromptTemplate, LLMChain

prompt_template = """
        ### 指示:\n{instruction}\n\n### 質問:\n{input}\n\n### 応答:
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["instruction", "input"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

input = "中国の歴史に関する論文のタイトルを一つ紹介して"

source = ""
results_with_scores = vectorstore.similarity_search_with_score(input)

for doc, score in results_with_scores:
    abst = "概要: " + doc.page_content + " "
    metadata = doc.metadata
    article = "記事タイトル: " + metadata["article_title"] + " "
    #print(article)
    link = "リンク: " + metadata["link"] + " "
    row = article + link + abst
    source += "\n" + row
#print(source)
instruction = "質問に対して以下の文献を参考にして、答えを作成してください。" + source

result = llm_chain.invoke({"instruction": instruction, "input": input})

print(result["text"])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


質問に対する答えは、「啓に関する基礎的考察」です。この論文は、日本の史料中の「啓」という語句の分析を手懸りに、中国における啓との比較を通じて検討したものです。


## Swallowのプロンプトを書き換えてつかう

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


model_name = "tokyotech-llm/Swallow-7b-instruct-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

In [ ]:

prompt_template = (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"

    )

input = "中国の歴史に関する論文を一つ紹介してください。またその論文を短く要約してください"

source = ""
results_with_scores = vectorstore.similarity_search_with_score(input)

for doc, score in results_with_scores:
    abst = "概要: " + doc.page_content + " "
    metadata = doc.metadata
    article = "記事タイトル: " + metadata["article_title"] + " "
    #print(article)
    link = "リンク: " + metadata["link"] + " "
    row = article + link + abst
    source += "\n" + row
#print(source)
instruction = "質問に対して以下の文献を参考にして、答えを作成してください。存在する論文のみを挙げてください" + source


prompt = prompt_template.format(instruction=instruction, input=input)
#print(prompt)

In [ ]:
input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=500,
    temperature=0.01,
    top_p=0.95,
    do_sample=True,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
質問に対して以下の文献を参考にして、答えを作成してください。存在する論文のみを挙げてください
記事タイトル: 啓に関する基礎的考察 リンク: https://doi.org/10.24471/shigaku.131.10_1 概要: 本稿は、七・八世紀の日本における啓という文書様式について、史料中の「啓」という語句の分析を手懸りに、中国における啓との比較を通じて検討したものである。 まず日本の史料をみると、「啓」は『日本書紀』本文や風土記では天皇以外に、『書紀』分註や天寿国繡帳銘では天皇にのみに用いる語句であり、ここからそれぞれの成立時期における啓の機能に差違がみいだせる。次に中国の啓は、六朝時代では公私問わず皇帝をも対象にできる汎用の上申文書（六朝的な啓）であり、唐代では皇帝以外に奉ずる文書（唐的な啓）であった。これにより、日本で天皇に「啓」を用いるのは六朝的な啓の影響で、天皇に「啓」を用いないのは唐的な啓の影響と考えられる。つまり、日本では六朝から唐的な啓へという転換が起きていたのである。また、東アジアにおける八世紀の啓をみると、啓は君主以外に奉ずる文書と言える。その機能に従って渤海は日本に啓を用いたが、日本は啓の機能を知りながら強引に渤海を朝貢国とする対応をとっており、日本の対渤海外交の方針が窺える。 　また、七世紀の文書木簡における啓は汎用の上申文書としての性格が窺え、これも六朝的な啓の影響が考えられる。啓は当時の状況に適した文書様式として受容され、文書行政を展開するための先駆的な役割を果たしていた。 このように七世紀は六朝的な啓が用いられていたが、大宝令施行によって唐的な啓への転換が起こり、公式令啓式条などにみえるように、啓は天皇以外に奉ずる文書となった。一方、公式令の範囲外では六朝の要素が残存しており、唐の啓と完全に同質となったわけでは無かった。とはいえ、啓の転換の結果、天皇に上申するための文書様式は、表・奏という天皇に上申するため「だけ」の文書様式となり、啓の転換は文書様式上における天皇の地位の可視化を意図していたと言えよう。 
記事タイトル: 日中戦争の開始前後における国策研究会と大蔵公望